In [1]:
import argparse
import json

In [1]:
# for running the code in the notebook
%load_ext autoreload
%autoreload 2
%aimport RNN_model, RNN_apply_ind

# Data Validation

In [3]:
'''
If not have run before
'''
import RNN_apply_ind, os, json, argparse

folder_path = r'C:\Users\qilei.zhang\OneDrive - Frontier Airlines\Documents\Data\USconti'
seats_file_name = r'\Schedule_Monthly_Summary_Report_Conti.csv'
perf_file_name = r'\Airline_Performance_Report_Conti.csv'
apply_file_name = '\Schedule_Monthly_Summary_2023Q1234.csv'
# Load parameters from the JSON file.
if not os.path.exists('parameters.json'):
    print("parameters.json does not exist, Find the file and put it in the same folder as this file")
with open('parameters.json', 'r') as f:
    args = argparse.Namespace(**json.load(f))

RNN_apply_ind.main_apply(args, folder_path, seats_file_name, perf_file_name, apply_file_name)

Using the provided arguments.
-------- Start ----------
Will use cpu as device
Missing airports database loaded.
Major hubs database loaded.
Dimension mapping loaded.
Applying data exists, will load it
Train Boundary quarter (<): Q1 2023
Validation Boundary quarter (>): Q1 2020
Test data (>): Q4 2020
Original data loaded.
Main scaler rebuilt.
Seat scaler rebuilt.
Validation/Test data scaled.
Date features created.
Validation data prepared.
Model loaded
Evaluating: batch 100 of 598
Evaluating: batch 200 of 598
Evaluating: batch 300 of 598
Evaluating: batch 400 of 598
Evaluating: batch 500 of 598
            MAE          RMSE        MAPE           std       <1%       <5%  \
0  44836.953125  46412.007812  322.719513  12447.246094  0.000424  0.000849   
1  48362.109375  49592.316406  295.274170  12919.898438  0.000000  0.000212   
2  43448.089844  45295.800781  285.345886   7254.037109  0.000000  0.000424   

       <10%      <20%  
0  0.001909  0.006152  
1  0.000424  0.001485  
2  0.0021

In [7]:
'''
quick look at the result
'''
import pandas as pd
from RNN_apply_ind import DataAna

apply_filename='./data/applying_data.csv'
ana_df_name = "./results/data_to_ana_apply.csv"
orig_df = pd.read_csv(apply_filename)

ana = DataAna(ana_df_name)
ana.merge_previous_data(orig_df)

while True:
    user_input = input("Enter airline and route, separated by comma, or 'c' to exit: ")
    if user_input.lower() == 'c':
        break
    try:
        airline, route = user_input.split(',')
        airline = airline.strip()  # remove possible leading/trailing whitespaces
        route = route.strip()  # remove possible leading/trailing whitespaces
        ana.plot_prediction(airline, route)
    except ValueError:
        print("Invalid input, please enter the airline and route separated by a comma or 'continue' to proceed.")

# Add new performance data and seats data to original data

In [14]:
'''
Add new performance data and seats data to original data
'''

import pandas as pd

folder_path = r'C:\Users\qilei.zhang\OneDrive - Frontier Airlines\Documents\Data\USconti'
seats_file_name = r'\Schedule_Monthly_Summary_Report_Conti.csv'
perf_file_name = r'\Airline_Performance_Report_Conti.csv'

seats_file_new = r'\Schedule_Monthly_Summary_2023Q1.csv'
perf_file_new = r'\Airline_Performance_Report_USconti_2023Q1.csv'

seats_df = pd.read_csv(folder_path + seats_file_name)
perf_df = pd.read_csv(folder_path + perf_file_name)

seats_df_new = pd.read_csv(folder_path + seats_file_new)
perf_df_new = pd.read_csv(folder_path + perf_file_new)


# Check the column names are the same othewise not proceed
if not (seats_df.columns == seats_df_new.columns).all():
    print("Column names are not the same, please check the data")
    raise ValueError

if not (perf_df.columns == perf_df_new.columns).all():
    print("Column names are not the same, please check the data")
    raise ValueError

# Check the data types are the same othewise not proceed
if not (seats_df.dtypes == seats_df_new.dtypes).all():
    print("Column types are not the same, please check the data")
    raise ValueError

if not (perf_df.dtypes == perf_df_new.dtypes).all():
    print("Column types are not the same, please check the data")
    raise ValueError

seats_df = pd.concat([seats_df, seats_df_new])
perf_df = pd.concat([perf_df, perf_df_new])

seats_df.to_csv(folder_path + seats_file_name, index=False)
perf_df.to_csv(folder_path + perf_file_name, index=False)


# Test if the dates are correct

In [1]:

from RNN_model import calculate_quarters

calculate_quarters(pred_num_quarters=3, seq_num=10, start_quarter="Q1 2023", skip_quarters=0)

Train Boundary quarter (<): Q2 2022
Test data (>): Q4 2020


('Q2 2022', 'Q1 2020', 'Q4 2020')

# Automatically tune the hyperparameters and record the results

In [27]:

'''
Create combinations of parameters to tune
'''

# Change the parameters.json file to have the following
import pandas as pd
import itertools

params_to_tune = {
    "learning_rate": [1e-03, 1e-04],
    # "momentum": [0.95, 0.98],
    # "batch_size": [64, 128],
    "epochs": [20, 30],
    # "n_layers": [4, 5],
    "drop_prob": [0.35, 0.4],
    "bidirectional": [True, False], 
    "if_skip": [True, False], 
    # "if_feed_drop": [True, False], 
    # "if_feed_norm": [True, False],
}

# Generate all combinations
keys, values = zip(*params_to_tune.items())
param_combinations = [dict(zip(keys, v)) for v in itertools.product(*values)]

# Create DataFrame
df = pd.DataFrame(param_combinations)
# df

In [ ]:
# Change the parameters.json file to have the following
import pandas as pd
import itertools

params_to_tune = {
    "learning_rate": [1e-03, 1e-04],
    # "momentum": [0.95, 0.98],
    # "batch_size": [64, 128],
    "epochs": [30],
    # "n_layers": [4, 5],
    "drop_prob": [0.35, 0.4],
    "bidirectional": [True], 
    "if_skip": [True], 
    "if_feed_drop": [True, False], 
    "if_feed_norm": [True, False],
}

# Generate all combinations
keys, values = zip(*params_to_tune.items())
param_combinations = [dict(zip(keys, v)) for v in itertools.product(*values)]

# Create DataFrame
df = pd.DataFrame(param_combinations)
# df

In [22]:
# Load parameters from the JSON file. Check if the parameters are loaded correctly.
with open('parameters.json', 'r') as f:
    args = argparse.Namespace(**json.load(f))
args

Namespace(resume_training=False, MSE_or_GaussianNLLLoss='GaussianNLLLoss', pred_num_quarters=3, seq_num=10, if_add_time_info=False, learning_rate=0.0001, momentum=0.95, batch_size=64, epochs=20, num_workers=1, shuffle=True, fixed_seed=True, rnn_type='LSTM', n_layers=4, drop_prob=0.35, num_heads=6, start_year=2004, checkpoint_file_name='checkpoint_20.pth', bidirectional=True, if_skip=False, if_feed_drop=True, if_feed_norm=True, start_quarter='Q1 2023', skip_quarters=2, validation_type='Val', tune=False)

In [21]:
import RNN_model
import RNN_apply_ind, os

folder_path = r'C:\Users\qilei.zhang\OneDrive - Frontier Airlines\Documents\Data\USconti'
seats_file_name = r'\Schedule_Monthly_Summary_Report_Conti.csv'
perf_file_name = r'\Airline_Performance_Report_Conti.csv'
apply_file_name = 'Schedule_Monthly_Summary_2023Q1234.csv'

for idx, row in df.iterrows():
    print(idx)
    # Extract parameters from the row
    params = row.to_dict()

    # Load parameters from the JSON file.
    with open('parameters.json', 'r') as f:
        args = argparse.Namespace(**json.load(f))

    # assign the all parameters from the row to the args object
    for key, value in params.items():
        setattr(args, key, value)
        print(key, value)

    # Check types of certain parameters, and transform them if necessary
    if not isinstance(args.epochs, int):
        args.epochs = int(args.epochs)
    if not isinstance(args.batch_size, int):
        args.batch_size = int(args.batch_size)
    if not isinstance(args.n_layers, int):
        args.n_layers = int(args.n_layers)

    # Run the model
    # Run Training
    RNN_model.main_program(args, folder_path, seats_file_name, perf_file_name, tune_folder=str(idx))

    # Run validation
    RNN_apply_ind.main_apply(args, folder_path, seats_file_name, perf_file_name, apply_file_name, tune_folder=str(idx))


0
learning_rate 0.001
epochs 20
drop_prob 0.35
bidirectional True
if_skip True
1
learning_rate 0.001
epochs 20
drop_prob 0.35
bidirectional True
if_skip False
2
learning_rate 0.001
epochs 20
drop_prob 0.35
bidirectional False
if_skip True
3
learning_rate 0.001
epochs 20
drop_prob 0.35
bidirectional False
if_skip False
4
learning_rate 0.001
epochs 20
drop_prob 0.4
bidirectional True
if_skip True
5
learning_rate 0.001
epochs 20
drop_prob 0.4
bidirectional True
if_skip False
6
learning_rate 0.001
epochs 20
drop_prob 0.4
bidirectional False
if_skip True
7
learning_rate 0.001
epochs 20
drop_prob 0.4
bidirectional False
if_skip False
8
learning_rate 0.001
epochs 30
drop_prob 0.35
bidirectional True
if_skip True
9
learning_rate 0.001
epochs 30
drop_prob 0.35
bidirectional True
if_skip False
10
learning_rate 0.001
epochs 30
drop_prob 0.35
bidirectional False
if_skip True
11
learning_rate 0.001
epochs 30
drop_prob 0.35
bidirectional False
if_skip False
12
learning_rate 0.001
epochs 30
drop_prob

# Download files in batch from Azure Databricks

In [2]:
import os
import subprocess

# define the remote path in databricks and the local download path
remote_base_path = 'dbfs:/FileStore/SeatPre/RunModelTune3/'
local_base_path = 'C:\\Users\\qilei.zhang\\Downloads\\'

# define the subfolder and file structure
folder_structure = {
    "model": ["model.pth"],
    "results": ["error_table_apply.csv", "best_route.csv", "data_to_ana_apply.csv"],
    "checkpoint": ["checkpoint_30.pth"],
}

# iterate over each numbered folder
for i in range(10):  # replace with the maximum numbered folder
    for folder, files in folder_structure.items():
        # remote_folder_path = os.path.join(remote_base_path, str(i), folder)
        remote_folder_path = remote_base_path + str(i) + '/' + folder + '/'
        local_folder_path = os.path.join(local_base_path, str(i), folder)
        # create the local directory if it doesn't exist
        os.makedirs(local_folder_path, exist_ok=True)

        # iterate over each file in the folder
        for file in files:
            # remote_file_path = os.path.join(remote_folder_path, file)
            remote_file_path = remote_folder_path + file
            local_file_path = os.path.join(local_folder_path, file)

            # print the file path
            print(remote_file_path, ' ------ ', local_folder_path)

            # check if the file already exists locally
            if os.path.exists(local_file_path):
                print('file already exists, skipping')
                continue

            # execute the databricks fs cp command
            subprocess.run(['databricks', 'fs', 'cp', remote_file_path, local_folder_path], check=True)
            print('done')

### Copy preprocessed data to the local directory

In [4]:
import os
import subprocess

# define the remote path in databricks and the local download path
remote_base_path = 'dbfs:/FileStore/SeatPre/RunModelTune3/'
local_base_path = 'C:\\Users\\qilei.zhang\\Downloads\\para_tune3\\'

# define the subfolder and file structure
folder_structure = {
    "data": ["applying_data.csv", "cat_mapping.pkl", "embed_dim_mapping.pkl", 
             "le_airlines.pkl", "le_airports.pkl", "scaled_data.csv", "testing_data.csv",
             "training_data.csv"],
}

# iterate over each numbered folder
for folder, files in folder_structure.items():
    # remote_folder_path = os.path.join(remote_base_path, str(i), folder)
    remote_folder_path = remote_base_path + folder + '/'
    local_folder_path = os.path.join(local_base_path, folder)
    # create the local directory if it doesn't exist
    os.makedirs(local_folder_path, exist_ok=True)

    # iterate over each file in the folder
    for file in files:
        # remote_file_path = os.path.join(remote_folder_path, file)
        remote_file_path = remote_folder_path + file
        local_file_path = os.path.join(local_folder_path, file)

        # print the file path
        print(remote_file_path, ' ------ ', local_folder_path)

        # check if the file already exists locally
        if os.path.exists(local_file_path):
            print('file already exists, skipping')
            continue

        # execute the databricks fs cp command
        subprocess.run(['databricks', 'fs', 'cp', remote_file_path, local_folder_path], check=True)
        print('done')

dbfs:/FileStore/SeatPre/RunModelTune3/data/applying_data.csv  ------  C:\Users\qilei.zhang\Downloads\para_tune3\data
done
dbfs:/FileStore/SeatPre/RunModelTune3/data/cat_mapping.pkl  ------  C:\Users\qilei.zhang\Downloads\para_tune3\data
done
dbfs:/FileStore/SeatPre/RunModelTune3/data/embed_dim_mapping.pkl  ------  C:\Users\qilei.zhang\Downloads\para_tune3\data
done
dbfs:/FileStore/SeatPre/RunModelTune3/data/le_airlines.pkl  ------  C:\Users\qilei.zhang\Downloads\para_tune3\data
done
dbfs:/FileStore/SeatPre/RunModelTune3/data/le_airports.pkl  ------  C:\Users\qilei.zhang\Downloads\para_tune3\data
done
dbfs:/FileStore/SeatPre/RunModelTune3/data/scaled_data.csv  ------  C:\Users\qilei.zhang\Downloads\para_tune3\data
done
dbfs:/FileStore/SeatPre/RunModelTune3/data/testing_data.csv  ------  C:\Users\qilei.zhang\Downloads\para_tune3\data
done
dbfs:/FileStore/SeatPre/RunModelTune3/data/training_data.csv  ------  C:\Users\qilei.zhang\Downloads\para_tune3\data
done


# Run current forecast

In [3]:
import RNN_apply_ind, os, json, argparse

folder_path = r'C:\Users\qilei.zhang\OneDrive - Frontier Airlines\Documents\Data\USconti'
seats_file_name = r'\Schedule_Monthly_Summary_Report_Conti.csv'
perf_file_name = r'\Airline_Performance_Report_Conti.csv'
apply_file_name = '\Schedule_Monthly_Summary_2023Q1234.csv'
# Load parameters from the JSON file.
if not os.path.exists('parameters.json'):
    print("parameters.json does not exist, Find the file and put it in the same folder as this file")
with open('parameters.json', 'r') as f:
    args = argparse.Namespace(**json.load(f))

key = "validation_type"
value = "test"
setattr(args, key, value)

RNN_apply_ind.main_apply(args, folder_path, seats_file_name, perf_file_name, apply_file_name)

Using the provided arguments.
-------- Start ----------
Will use cpu as device
Missing airports database loaded.
Major hubs database loaded.
Dimension mapping loaded.
Applying data exists, will load it
Train Boundary quarter (<): Q1 2023
Validation Boundary quarter (>): Q1 2020
Test data (>): Q4 2020
Original data loaded.
Main scaler rebuilt.
Seat scaler rebuilt.
Validation/Test data scaled.
Date features created.
Validation data prepared.
Model loaded
Evaluating: batch 100 of 695
Evaluating: batch 200 of 695
Evaluating: batch 300 of 695
Evaluating: batch 400 of 695
Evaluating: batch 500 of 695
Evaluating: batch 600 of 695
There is 5554 routes in the route dictionary
Time used: 1.2593866507212321 minutes
-------- End ----------


# Playground

In [20]:
import pandas as pd

class QuarterFilling:
    def __init__(self, seq_len, n_future, skip_quarters):
        self.seq_len = seq_len
        self.n_future = n_future
        self.skip_quarters = skip_quarters

    def fill_missing_quarters(self, route_df):
        # Convert 'Date' to a string format for consistency
        route_df['Date'] = route_df['Date'].astype(str)
        quarters = ["Q1", "Q2", "Q3", "Q4"]

        # Define a function to generate the next quarter
        def next_quarter(q, y):
            if q == "Q4":
                return "Q1", y + 1
            else:
                next_q = quarters[quarters.index(q) + 1]
                return next_q, y

        # Generate a list of all quarters present in route_df
        all_dates = list(route_df['Date'])
        missing_quarters = []

        # Find missing quarters but not for the last few records
        for i in range(len(all_dates) - self.n_future - self.skip_quarters - 1):
            q, y = all_dates[i].split()
            y = int(y)
            next_q, next_y = next_quarter(q, y)

            if f"{next_q} {next_y}" != all_dates[i + 1]:
                missing_quarters.append(f"{next_q} {next_y}")

        # Fill in the missing quarters
        for missing in missing_quarters:
            q, y = missing.split()
            zero_row = pd.Series({col: -1e10 for col in route_df.columns}, name=missing)
            zero_row['Date'] = missing
            # Set all quarter columns to False
            for i in [1, 2, 3, 4]:
                zero_row[f'quarter_{i}'] = False
            zero_row[f'quarter_{quarters.index(q) + 1}'] = True  # Set the correct quarter to true
            # route_df = route_df.append(zero_row, ignore_index=True)
            route_df = pd.concat([route_df, pd.DataFrame([zero_row])]).reset_index(drop=True)

        # Sort by Date after adding missing quarters
        route_df = route_df.sort_values(by="Date").reset_index(drop=True)

        # If length is still less, prepend rows with zeros for previous quarters
        while len(route_df) < (self.seq_len + self.n_future + self.skip_quarters):
            first_q, first_y = route_df.iloc[0]['Date'].split()
            first_y = int(first_y)

            if first_q == "Q1":
                prev_q, prev_y = "Q4", first_y - 1
            else:
                prev_q = quarters[quarters.index(first_q) - 1]
                prev_y = first_y

            zero_row = pd.Series({col: -1e10 for col in route_df.columns}, name=f"{prev_q} {prev_y}")
            zero_row['Date'] = f"{prev_q} {prev_y}"
            for i in [1, 2, 3, 4]:
                zero_row[f'quarter_{i}'] = False
            zero_row[f'quarter_{quarters.index(prev_q) + 1}'] = True  # Set the correct quarter to 1
            route_df = pd.concat([pd.DataFrame([zero_row]), route_df]).reset_index(drop=True)

        return route_df



In [21]:
route_df = pd.read_csv('route_df.csv', index_col=0)
route_df

,Orig,Dest,Date,Miles,Deps/Day,Seats/Day,Seats/Dep,Pax/Day,Pax/Dep,Load Factor,...,pand_year,num_comp,mkt_share,mkt_size,Date_delta,time_scaled,quarter_1,quarter_2,quarter_3,quarter_4
Mkt Al,,,,,,,,,,,,,,,,,,,,,
3E,ATL,MKL,Q1 2018,-0.943008,-1.006200,-0.874618,-2.062496,-0.835785,-1.762664,1.006096,...,1,-0.722980,0.776915,-0.795749,4733.856,0.736888,True,False,False,False
3E,ATL,MKL,Q2 2018,-0.943008,-0.802390,-0.859129,-2.028518,-0.820681,-1.744648,0.517668,...,1,-0.722980,0.776915,-0.789987,4811.616,0.749856,False,True,False,False
3E,ATL,MKL,Q3 2018,-0.943008,-0.778412,-0.861752,-2.062496,-0.820287,-1.753217,1.351743,...,1,-0.722980,0.776915,-0.789876,4890.240,0.762968,False,False,True,False
3E,ATL,MKL,Q4 2018,-0.943008,-0.790401,-0.862486,-2.062496,-0.820736,-1.750141,1.468035,...,1,-0.722980,0.776915,-0.789876,4969.728,0.776225,False,False,False,True
3E,ATL,MKL,Q1 2019,-0.943008,-0.778412,-0.862317,-2.066620,-0.821662,-1.762884,1.154046,...,1,-0.722980,0.776915,-0.790098,5049.216,0.789481,True,False,False,False
3E,ATL,MKL,Q3 2019,-0.943008,-0.778412,-0.861752,-2.062496,-0.820774,-1.756952,1.218653,...,1,-0.722980,0.776915,-0.789876,5205.600,0.815562,False,False,True,False
3E,ATL,MKL,Q4 2019,-0.943008,-1.006200,-0.874505,-2.062496,-0.835305,-1.754755,1.298765,...,2,-0.722980,0.776915,-0.789876,5285.088,0.828818,False,False,False,True
3E,ATL,MKL,Q1 2020,-0.943008,-0.794397,-0.862542,-2.062496,-0.826111,-1.791226,-0.041181,...,2,-0.722980,0.776915,-0.789987,5364.576,0.842075,True,False,False,False
3E,ATL,MKL,Q2 2020,-0.943008,-0.786405,-0.862289,-2.062496,-0.836587,-1.872737,-3.037003,...,2,-0.722980,0.776915,-0.789987,5443.200,0.855187,False,True,False,False


In [24]:

# Example usage
quarter_filler = QuarterFilling(seq_len=10, n_future=3, skip_quarters=2)
filled_route_df = quarter_filler.fill_missing_quarters(route_df)
filled_route_df['SortDate'] = filled_route_df['Date'].apply(lambda x: int(x.split(' ')[1]) * 4 + int(x.split(' ')[0][1]))
filled_route_df = filled_route_df.sort_values(by="SortDate").reset_index(drop=True)
filled_route_df

,Orig,Dest,Date,Miles,Deps/Day,Seats/Day,Seats/Dep,Pax/Day,Pax/Dep,Load Factor,...,num_comp,mkt_share,mkt_size,Date_delta,time_scaled,quarter_1,quarter_2,quarter_3,quarter_4,SortDate
0,0,0,Q4 2017,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000,0.000000,False,False,False,True,8072
1,ATL,MKL,Q1 2018,-0.943008,-1.006200,-0.874618,-2.062496,-0.835785,-1.762664,1.006096,...,-0.722980,0.776915,-0.795749,4733.856,0.736888,True,False,False,False,8073
2,ATL,MKL,Q2 2018,-0.943008,-0.802390,-0.859129,-2.028518,-0.820681,-1.744648,0.517668,...,-0.722980,0.776915,-0.789987,4811.616,0.749856,False,True,False,False,8074
3,ATL,MKL,Q3 2018,-0.943008,-0.778412,-0.861752,-2.062496,-0.820287,-1.753217,1.351743,...,-0.722980,0.776915,-0.789876,4890.240,0.762968,False,False,True,False,8075
4,ATL,MKL,Q4 2018,-0.943008,-0.790401,-0.862486,-2.062496,-0.820736,-1.750141,1.468035,...,-0.722980,0.776915,-0.789876,4969.728,0.776225,False,False,False,True,8076
5,ATL,MKL,Q1 2019,-0.943008,-0.778412,-0.862317,-2.066620,-0.821662,-1.762884,1.154046,...,-0.722980,0.776915,-0.790098,5049.216,0.789481,True,False,False,False,8077
6,0,0,Q2 2019,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000,0.000000,False,True,False,False,8078
7,ATL,MKL,Q3 2019,-0.943008,-0.778412,-0.861752,-2.062496,-0.820774,-1.756952,1.218653,...,-0.722980,0.776915,-0.789876,5205.600,0.815562,False,False,True,False,8079
8,ATL,MKL,Q4 2019,-0.943008,-1.006200,-0.874505,-2.062496,-0.835305,-1.754755,1.298765,...,-0.722980,0.776915,-0.789876,5285.088,0.828818,False,False,False,True,8080
9,ATL,MKL,Q1 2020,-0.943008,-0.794397,-0.862542,-2.062496,-0.826111,-1.791226,-0.041181,...,-0.722980,0.776915,-0.789987,5364.576,0.842075,True,False,False,False,8081


In [ ]:
fil